# ImSwitch Embedded Kernel Connection

This notebook connects to the running ImSwitch embedded kernel for hardware control and visualization.

**Prerequisites:**
1. Start ImSwitch with: `python -m imswitch --with-kernel`
2. Run the first cell to establish connection
3. Use the connected kernel for hardware control

In [11]:
# ImSwitch Kernel Connection Setup
import os
import json
import glob
from jupyter_client import BlockingKernelClient
from IPython.display import display, HTML
import sys
import time

def find_imswitch_kernel():
    """Find the ImSwitch embedded kernel connection file"""
    
    # Check common kernel locations
    kernel_dirs = [
        os.path.expanduser("~/Library/Jupyter/runtime"),  # macOS
        os.path.expanduser("~/.local/share/jupyter/runtime"),  # Linux
        os.path.expanduser("~/.jupyter/runtime"),  # Alternative location
    ]
    
    kernel_files = []
    for kernel_dir in kernel_dirs:
        if os.path.exists(kernel_dir):
            pattern = os.path.join(kernel_dir, "kernel-*.json")
            kernel_files.extend(glob.glob(pattern))
    
    if not kernel_files:
        print("❌ No kernel connection files found!")
        print("Make sure ImSwitch is running with: python -m imswitch --with-kernel")
        return None
    
    # Sort by modification time (newest first)
    kernel_files.sort(key=os.path.getmtime, reverse=True)
    
    print(f"📡 Found {len(kernel_files)} kernel(s):")
    for i, kf in enumerate(kernel_files[:5]):  # Show only first 5
        mtime = os.path.getmtime(kf)
        # Try to identify ImSwitch kernel by checking if it's recent and has the right pattern
        is_recent = (time.time() - mtime) < 3600  # Within last hour
        filename = os.path.basename(kf)
        marker = "🔥" if is_recent else "  "
        print(f"  {marker} {i+1}. {filename} (age: {time.time() - mtime:.0f}s)")
        
        # Show kernel info for recent ones
        if is_recent or i < 2:
            try:
                with open(kf, 'r') as f:
                    kernel_info = json.load(f)
                print(f"      kernel_name: '{kernel_info.get('kernel_name', 'EMPTY')}'")
                print(f"      ports: shell={kernel_info.get('shell_port')}, iopub={kernel_info.get('iopub_port')}")
            except:
                pass
    
    return kernel_files[0]  # Return newest

def test_kernel_connection(connection_file):
    """Test if a kernel connection file is for an active ImSwitch kernel"""
    
    try:
        with open(connection_file, 'r') as f:
            kernel_info = json.load(f)
        
        print(f"\n🔍 Testing connection to: {os.path.basename(connection_file)}")
        print(f"   Shell port: {kernel_info.get('shell_port')}")
        print(f"   Kernel name: '{kernel_info.get('kernel_name', 'EMPTY')}'")
        
        # Create kernel client with timeout
        client = BlockingKernelClient()
        client.load_connection_info(kernel_info)
        
        # Set shorter timeouts for testing
        client.session.timeout = 3
        
        try:
            client.start_channels()
            print("   📡 Channels started...")
            
            # Send a test command to check if it's ImSwitch
            msg_id = client.execute("print('Testing ImSwitch connection'); 'lasersManager' in globals()", silent=True)
            
            # Wait for reply with timeout
            reply = client.get_shell_msg(timeout=5)
            
            if reply['content']['status'] == 'ok':
                # Check if we got output indicating ImSwitch
                while True:
                    try:
                        msg = client.get_iopub_msg(timeout=2)
                        if msg['msg_type'] == 'execute_result':
                            result = msg['content']['data']['text/plain']
                            if 'True' in result:
                                print("   ✅ This is an active ImSwitch kernel!")
                                return client
                            elif 'False' in result:
                                print("   ❌ Kernel active but not ImSwitch (no lasersManager)")
                                client.stop_channels()
                                return None
                    except:
                        break
                        
                print("   ✅ Kernel is active (ImSwitch status unknown)")
                return client
            else:
                print(f"   ❌ Kernel returned error: {reply['content']}")
                
        except Exception as e:
            print(f"   ❌ Connection failed: {e}")
            
        try:
            client.stop_channels()
        except:
            pass
            
    except Exception as e:
        print(f"   ❌ Failed to read connection file: {e}")
        
    return None

def connect_to_imswitch_kernel():
    """Connect to the ImSwitch embedded kernel with better error handling"""
    
    connection_file = find_imswitch_kernel()
    if not connection_file:
        return None
    
    # Test the most recent kernels
    kernel_dirs = [
        os.path.expanduser("~/Library/Jupyter/runtime"),
        os.path.expanduser("~/.local/share/jupyter/runtime"),
    ]
    
    all_kernels = []
    for kernel_dir in kernel_dirs:
        if os.path.exists(kernel_dir):
            pattern = os.path.join(kernel_dir, "kernel-*.json")
            all_kernels.extend(glob.glob(pattern))
    
    # Sort by modification time and test recent ones
    all_kernels.sort(key=os.path.getmtime, reverse=True)
    recent_kernels = [k for k in all_kernels if (time.time() - os.path.getmtime(k)) < 3600][:5]
    
    print(f"\n🔍 Testing {len(recent_kernels)} recent kernel(s) for ImSwitch...")
    
    for kernel_file in recent_kernels:
        client = test_kernel_connection(kernel_file)
        if client:
            return client
    
    print("\n❌ No active ImSwitch kernel found!")
    print("💡 Solutions:")
    print("1. Make sure ImSwitch is running with: python -m imswitch --with-kernel")
    print("2. Try the manual connection method below")
    print("3. Check that the kernel hasn't crashed (look at ImSwitch console)")
    
    return None

# Establish connection
print("🔌 Connecting to ImSwitch embedded kernel...")
kernel_client = connect_to_imswitch_kernel()

if kernel_client:
    display(HTML('<div style="background: #d4edda; border: 1px solid #c3e6cb; padding: 10px; border-radius: 5px;"><strong>✅ Connection established!</strong><br>You can now execute commands in the ImSwitch kernel context.</div>'))
else:
    display(HTML('<div style="background: #f8d7da; border: 1px solid #f5c6cb; padding: 10px; border-radius: 5px;"><strong>❌ Connection failed!</strong><br>Use the manual connection method below.</div>'))

🔌 Connecting to ImSwitch embedded kernel...
📡 Found 591 kernel(s):
  1. kernel-v3c37ea4a13b47e8071f1b4f090c25b27d21252f12.json (modified: 1757504037.7949023)
  2. kernel-90884.json (modified: 1757503695.2400434)
  3. kernel-a6f2dd1b-4e98-4eee-84ac-e7b9f76c2154.json (modified: 1757500901.7092767)
  4. kernel-48828.json (modified: 1757500893.3621626)
  5. kernel-9a72f8a1-b075-47c9-b2a7-35c9eba85163.json (modified: 1757500853.1878245)
❌ Failed to connect: 


In [ ]:
# Manual Kernel Connection - Most Reliable Method
# This method manually connects this notebook to the ImSwitch kernel

def show_connection_instructions():
    """Show detailed instructions for manual kernel connection"""
    
    # Find latest kernel file
    kernel_dirs = [
        os.path.expanduser("~/Library/Jupyter/runtime"),  # macOS
        os.path.expanduser("~/.local/share/jupyter/runtime"),  # Linux
    ]
    
    kernel_files = []
    for kernel_dir in kernel_dirs:
        if os.path.exists(kernel_dir):
            pattern = os.path.join(kernel_dir, "kernel-*.json")
            kernel_files.extend(glob.glob(pattern))
    
    if kernel_files:
        # Get recent kernels (last hour)
        recent_kernels = []
        for kf in kernel_files:
            age = time.time() - os.path.getmtime(kf)
            if age < 3600:  # Last hour
                recent_kernels.append((kf, age))
        
        recent_kernels.sort(key=lambda x: x[1])  # Sort by age
        
        print("🔧 MANUAL CONNECTION METHOD (Recommended):")
        print("="*60)
        
        print("\n📋 Step-by-Step Instructions:")
        print("1. In Jupyter Notebook/Lab menu: Kernel → Change Kernel...")
        print("2. Select 'Existing Jupyter Server...'")
        print("3. Copy and paste this connection info:")
        
        if recent_kernels:
            latest_kernel, age = recent_kernels[0]
            try:
                with open(latest_kernel, 'r') as f:
                    kernel_info = json.load(f)
                
                print("\n" + "="*50)
                print("📋 CONNECTION INFO TO COPY:")
                print("="*50)
                connection_json = {
                    "shell_port": kernel_info.get('shell_port'),
                    "iopub_port": kernel_info.get('iopub_port'), 
                    "stdin_port": kernel_info.get('stdin_port'),
                    "control_port": kernel_info.get('control_port'),
                    "hb_port": kernel_info.get('hb_port'),
                    "ip": kernel_info.get('ip', '127.0.0.1'),
                    "key": kernel_info.get('key'),
                    "transport": kernel_info.get('transport', 'tcp'),
                    "signature_scheme": kernel_info.get('signature_scheme', 'hmac-sha256'),
                    "kernel_name": "imswitch"  # Give it a name for identification
                }
                
                print(json.dumps(connection_json, indent=2))
                print("="*50)
                
                print(f"\n? Kernel file: {os.path.basename(latest_kernel)}")
                print(f"🔸 Age: {age:.0f} seconds")
                print(f"🔸 Expected kernel_name: imswitch")
                
                # Alternative method
                print(f"\n🔄 ALTERNATIVE METHOD:")
                print(f"4. Or directly use this kernel file path:")
                print(f"   {latest_kernel}")
                print(f"\n5. Or use command line:")
                print(f"   jupyter console --existing {os.path.basename(latest_kernel)}")
                
                # Show recent kernels for comparison
                if len(recent_kernels) > 1:
                    print(f"\n📊 Other recent kernels ({len(recent_kernels)-1}):")
                    for kf, age in recent_kernels[1:4]:  # Show up to 3 more
                        print(f"   {os.path.basename(kf)} (age: {age:.0f}s)")
                        
            except Exception as e:
                print(f"Could not read kernel info: {e}")
                print(f"Try using kernel file directly: {latest_kernel}")
        else:
            print("❌ No recent kernels found!")
            print("Make sure ImSwitch is running with: python -m imswitch --with-kernel")
            
        print(f"\n💡 TROUBLESHOOTING:")
        print("- If connection fails, make sure ImSwitch is still running")
        print("- Look for 'ImSwitch' in the connection dialog title after connecting")
        print("- Test connection by running: test_imswitch_connection() (cell below)")
        print("- If kernel_name is empty, that's normal for embedded kernels")
            
    else:
        print("❌ No kernel files found. Start ImSwitch with: python -m imswitch --with-kernel")

show_connection_instructions()

## Solution for Empty kernel_name Issue

**The Problem:** ImSwitch embedded kernel shows `"kernel_name": ""` (empty), making it hard to identify in Jupyter.

**The Solution:** Use the manual connection method above with these steps:

### 🎯 Recommended Approach:

1. **In ImSwitch console, run:**
   ```python
   %connect_info
   ```

2. **Copy the JSON output** (like you did above)

3. **In this notebook:** 
   - Go to: **Kernel → Change Kernel → Existing Jupyter Server**
   - **Paste the complete JSON** from step 2
   - **Modify `kernel_name`** from `""` to `"imswitch"` for identification
   - Click **Connect**

4. **Verify connection** by running the test cell below

### 💡 Why This Works:
- Empty `kernel_name` is normal for embedded kernels
- Manual connection bypasses kernel discovery issues  
- Direct port connection is more reliable than automatic detection
- You can give it a custom name for identification

### 🔧 Alternative if JSON method fails:
- Use the kernel file path directly: `kernel-90884.json`
- In Jupyter: **Kernel → Change Kernel → Select kernel file**
- Browse to: `~/Library/Jupyter/runtime/kernel-90884.json`

In [13]:
# Test if we're connected to ImSwitch by checking for ImSwitch variables
def test_imswitch_connection():
    """Test if we're connected to the ImSwitch kernel"""
    
    print(f"🔍 ImSwitch Connection Test:")
    print("="*40)
    
    # Check for ImSwitch-specific variables
    imswitch_vars = [
        'moduleMainControllers',
        'master_controller', 
        'detectorsManager',
        'lasersManager',
        'stageManager'
    ]
    
    found_vars = []
    missing_vars = []
    
    for var in imswitch_vars:
        if var in globals():
            found_vars.append(var)
        else:
            missing_vars.append(var)
    
    if found_vars:
        print(f"✅ Found ImSwitch variables: {found_vars}")
        print("🎉 You are connected to the ImSwitch kernel!")
        
        # Test functionality of found managers
        print("\n🧪 Testing manager functionality:")
        try:
            if 'lasersManager' in found_vars:
                lasers = lasersManager.getAllDeviceNames()
                print(f"   🔦 Lasers: {list(lasers)} ({len(lasers)} devices)")
            
            if 'detectorsManager' in found_vars:
                detectors = list(detectorsManager.getAllDeviceNames())
                print(f"   📷 Detectors: {detectors} ({len(detectors)} devices)")
                
            if 'stageManager' in found_vars:
                stages = stageManager.getAllDeviceNames()
                print(f"   ? Stages: {list(stages)} ({len(stages)} devices)")
                
        except Exception as e:
            print(f"   ⚠️  Some managers exist but have errors: {e}")
            
        return True
        
    else:
        print(f"❌ Missing ImSwitch variables: {missing_vars}")
        print("❓ You might not be connected to the ImSwitch kernel.")
        
        # Check what kernel we're actually connected to
        print(f"\n? Current kernel info:")
        try:
            # Try to get some info about current environment
            import sys
            import os
            print(f"   Python: {sys.executable}")
            print(f"   PID: {os.getpid()}")
            
            # Check if we have any imswitch modules loaded
            imswitch_modules = [name for name in sys.modules.keys() if 'imswitch' in name.lower()]
            if imswitch_modules:
                print(f"   ImSwitch modules loaded: {len(imswitch_modules)}")
                print(f"   Examples: {imswitch_modules[:3]}")
            else:
                print(f"   No ImSwitch modules detected")
                
            # Check working directory
            print(f"   Working dir: {os.getcwd()}")
            
        except Exception as e:
            print(f"   Could not get kernel info: {e}")
        
        print(f"\n?💡 Solutions:")
        print("1. Make sure ImSwitch is running with: python -m imswitch --with-kernel")
        print("2. Use the manual connection method above")
        print("3. Check the ImSwitch console for errors")
        print("4. Verify you selected the correct kernel in the connection dialog")
        print("5. Try restarting ImSwitch and reconnecting")
        
        return False

# Test connection
print("Testing connection to ImSwitch kernel...")
is_connected = test_imswitch_connection()

if is_connected:
    display(HTML('<div style="background: #d4edda; border: 1px solid #c3e6cb; padding: 10px; border-radius: 5px; margin: 10px 0;"><strong>🎉 Successfully connected to ImSwitch!</strong><br>You can now control hardware and run experiments.</div>'))
else:
    display(HTML('<div style="background: #fff3cd; border: 1px solid #ffeaa7; padding: 10px; border-radius: 5px; margin: 10px 0;"><strong>⚠️ Not connected to ImSwitch kernel</strong><br>Use the manual connection method above to connect.</div>'))

🔍 ImSwitch Connection Test:
❌ Missing ImSwitch variables: ['moduleMainControllers', 'master_controller', 'detectorsManager', 'lasersManager', 'stageManager']
❓ You might not be connected to the ImSwitch kernel.

💡 Solutions:
1. Make sure ImSwitch is running with: python -m imswitch --with-kernel
2. Change kernel to ImSwitch kernel (see instructions above)
3. Or execute the connection code in the previous cells


## Available ImSwitch Variables

Once connected to the ImSwitch kernel, you have access to:

### Core Objects
- `moduleMainControllers` - Dictionary of all module controllers
- `master_controller` - ImControl's master controller
- `config` - ImSwitch configuration object

### Hardware Managers
- `detectorsManager` - Camera/detector control
- `lasersManager` - Laser control  
- `stageManager` - Stage/positioning control
- `LEDsManager` - LED control
- `recordingManager` - Recording control

### Quick Test Commands

In [5]:
# Quick test of ImSwitch managers (only run if connected)
try:
    # Test laser manager
    if 'lasersManager' in globals():
        laser_devices = lasersManager.getAllDeviceNames()
        print(f"🔦 Available lasers: {laser_devices}")
    
    # Test detector manager  
    if 'detectorsManager' in globals():
        detector_names = list(detectorsManager.getAllDeviceNames())
        print(f"📷 Available detectors: {detector_names}")
    
    # Test stage manager
    if 'stageManager' in globals():
        stage_devices = stageManager.getAllDeviceNames()
        print(f"🎯 Available stages: {stage_devices}")
        
    print("\n✅ ImSwitch managers are accessible!")
    
except NameError as e:
    print(f"❌ ImSwitch variables not found: {e}")
    print("Make sure you're connected to the ImSwitch kernel (see cells above)")
except Exception as e:
    print(f"⚠️  Error testing managers: {e}")


✅ ImSwitch managers are accessible!


## Hardware Control Examples

Once connected, you can control ImSwitch hardware directly:

In [6]:
# Laser Control Example
try:
    if 'lasersManager' in globals():
        # Get available lasers
        lasers = lasersManager.getAllDeviceNames()
        print(f"Available lasers: {lasers}")
        
        if lasers:
            laser_name = lasers[0]  # Use first laser
            
            # Set laser power to 50%
            lasersManager.setpower(laser_name, 50)
            print(f"✅ Set {laser_name} power to 50%")
            
            # Get current power
            current_power = lasersManager.getPower(laser_name)
            print(f"Current {laser_name} power: {current_power}%")
            
        else:
            print("No lasers configured")
    else:
        print("❌ lasersManager not available - not connected to ImSwitch kernel")
        
except Exception as e:
    print(f"Error controlling laser: {e}")

❌ lasersManager not available - not connected to ImSwitch kernel


In [7]:
# Camera/Detector Control Example
import numpy as np
import matplotlib.pyplot as plt

try:
    if 'detectorsManager' in globals():
        # Get available detectors
        detectors = list(detectorsManager.getAllDeviceNames())
        print(f"Available detectors: {detectors}")
        
        if detectors:
            detector_name = detectors[0]  # Use first detector
            
            # Snap an image
            print(f"📸 Snapping image with {detector_name}...")
            image = detectorsManager.snap(detector_name)
            
            if image is not None:
                # Display image statistics
                print(f"Image shape: {image.shape}")
                print(f"Image dtype: {image.dtype}")
                print(f"Min intensity: {np.min(image)}")
                print(f"Max intensity: {np.max(image)}")
                print(f"Mean intensity: {np.mean(image):.1f}")
                
                # Plot image
                plt.figure(figsize=(10, 4))
                
                plt.subplot(1, 2, 1)
                plt.imshow(image, cmap='gray')
                plt.title(f'Captured Image ({detector_name})')
                plt.colorbar()
                
                plt.subplot(1, 2, 2)
                plt.hist(image.flatten(), bins=50, alpha=0.7)
                plt.title('Intensity Histogram')
                plt.xlabel('Intensity')
                plt.ylabel('Count')
                
                plt.tight_layout()
                plt.show()
                
            else:
                print("❌ Failed to capture image")
        else:
            print("No detectors configured")
    else:
        print("❌ detectorsManager not available - not connected to ImSwitch kernel")
        
except Exception as e:
    print(f"Error with detector: {e}")

❌ detectorsManager not available - not connected to ImSwitch kernel


In [8]:
# Stage Control Example  
try:
    if 'stageManager' in globals():
        # Get available stages
        stages = stageManager.getAllDeviceNames()
        print(f"Available stages: {stages}")
        
        if stages:
            stage_name = stages[0]  # Use first stage
            
            # Get current position
            current_pos = stageManager.getPosition(stage_name)
            print(f"Current position ({stage_name}): {current_pos}")
            
            # Move relative (example: move 10 units in X)
            print(f"Moving {stage_name} +10 units in X...")
            stageManager.moveRel(stage_name, {"X": 10})
            
            # Get new position
            new_pos = stageManager.getPosition(stage_name)
            print(f"New position ({stage_name}): {new_pos}")
            
        else:
            print("No stages configured")
    else:
        print("❌ stageManager not available - not connected to ImSwitch kernel")
        
except Exception as e:
    print(f"Error with stage: {e}")

❌ stageManager not available - not connected to ImSwitch kernel


## Live Monitoring Dashboard

Create a real-time dashboard for monitoring ImSwitch hardware:

In [9]:
# Interactive Hardware Dashboard
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import threading

class ImSwitchDashboard:
    def __init__(self):
        self.running = False
        self.output = widgets.Output()
        
        # Control widgets
        self.start_button = widgets.Button(description="Start Monitoring", button_style='success')
        self.stop_button = widgets.Button(description="Stop Monitoring", button_style='danger')
        self.refresh_rate = widgets.FloatSlider(value=1.0, min=0.5, max=5.0, step=0.5, 
                                               description='Refresh (s):')
        
        self.start_button.on_click(self.start_monitoring)
        self.stop_button.on_click(self.stop_monitoring)
        
        # Layout
        controls = widgets.HBox([self.start_button, self.stop_button, self.refresh_rate])
        self.dashboard = widgets.VBox([controls, self.output])
        
    def start_monitoring(self, button):
        self.running = True
        self.monitor_thread = threading.Thread(target=self.monitor_loop)
        self.monitor_thread.daemon = True
        self.monitor_thread.start()
        
    def stop_monitoring(self, button):
        self.running = False
        
    def monitor_loop(self):
        while self.running:
            with self.output:
                clear_output(wait=True)
                self.update_status()
            time.sleep(self.refresh_rate.value)
            
    def update_status(self):
        print(f"🕐 ImSwitch Status - {time.strftime('%H:%M:%S')}")
        print("="*50)
        
        try:
            # Laser status
            if 'lasersManager' in globals():
                lasers = lasersManager.getAllDeviceNames()
                print(f"🔦 Lasers ({len(lasers)}):")
                for laser in lasers:
                    try:
                        power = lasersManager.getPower(laser)
                        enabled = lasersManager.getLaserState(laser) if hasattr(lasersManager, 'getLaserState') else 'Unknown'
                        print(f"   {laser}: {power}% {'🟢' if enabled else '🔴'}")
                    except:
                        print(f"   {laser}: Status unavailable")
                        
            # Detector status
            if 'detectorsManager' in globals():
                detectors = list(detectorsManager.getAllDeviceNames())
                print(f"\n📷 Detectors ({len(detectors)}):")
                for detector in detectors:
                    try:
                        # Try to get some detector info
                        print(f"   {detector}: Ready")
                    except:
                        print(f"   {detector}: Status unavailable")
                        
            # Stage status  
            if 'stageManager' in globals():
                stages = stageManager.getAllDeviceNames()
                print(f"\n🎯 Stages ({len(stages)}):")
                for stage in stages:
                    try:
                        pos = stageManager.getPosition(stage)
                        pos_str = ", ".join([f"{k}:{v:.1f}" for k, v in pos.items()])
                        print(f"   {stage}: {pos_str}")
                    except:
                        print(f"   {stage}: Position unavailable")
                        
        except Exception as e:
            print(f"❌ Error updating status: {e}")
            print("Make sure you're connected to the ImSwitch kernel")
    
    def show(self):
        display(self.dashboard)

# Create and show dashboard
dashboard = ImSwitchDashboard()
dashboard.show()

## Troubleshooting

### Common Issues:

1. **"ImSwitch variables not found"**
   - Make sure ImSwitch is running with `--with-kernel` flag
   - Check that you've connected to the correct kernel

2. **"No kernel connection files found"**
   - ImSwitch might not be running with embedded kernel
   - Check the kernel runtime directory permissions

3. **"Connection timeouts"**
   - The kernel might be busy or crashed
   - Restart ImSwitch with `--with-kernel`

4. **"Manager methods not working"**
   - Hardware might not be properly configured
   - Check ImSwitch console for error messages

### Manual Connection Steps:

1. Start ImSwitch: `python -m imswitch --with-kernel`
2. In ImSwitch console: `%connect_info`
3. Copy the displayed connection info
4. In this notebook: Kernel → Change Kernel → Existing Jupyter Server
5. Paste connection info and connect